In [1]:
import pandas as pd
import numpy as np

- добавить google trends на автора (проверить на 1 авторе руками) 
- аналоги по боксплотам 
- поменять ящики ниша/канал и таблицы: психология, медицина, искусство
- trends: зубарева, суркова, вагин
- сезонность по месяцам прогнать
- тренд до/после: книга и дата старта/конца, продажи до/после (регрессия до и после, изменения в коэффициентах)

In [2]:
ndata = pd.read_csv('ExportRetail_utf8.csv', sep = ';')

C:\Users\1\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df = pd.read_excel('Продвижение.xlsx')

In [4]:
i = df['ISBN']
i.dropna(inplace = True)

In [5]:
pr = ['978-5-17-122673-2', '978-5-271-48520-6', '978-5-17-117233-6']
i = i[~i.isin(pr)]

In [6]:
i = i.to_numpy(dtype = str)
i = np.char.strip(i)

In [8]:
ndata1 = ndata[(ndata['Год'] == 2020) & (ndata['Тип_Данных'] != 'Отгрузки')]

In [7]:
ndata['Prom'] = ndata['ISBN'].apply(lambda x: 1 if x in i else 0)

C:\Users\1\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  """Entry point for launching an IPython kernel.


In [9]:
pd.set_option('display.max_columns', None)

In [10]:
books = pd.read_excel('Данные по книгам.xlsx')
b1 = books[['Код 1С', 'Ниша', 'Цена базовая', 'Автор', 'Наименование']]

In [11]:
b1.columns = ['1C', 'Ниша', 'Цена', 'Автор', 'Название']
ndata1.columns = ['1C', 'ISBN', 'EAN', 'Тип_Данных', 'Месяц', 'Год', 'Кол-во', 'Prom']

In [12]:
fdata = pd.merge(ndata1,
                 b1,
                 on = '1C',
                 how ='inner')

In [13]:
def month(x):
    if x in [1,2,3]:
        return 1
    elif x in [4,5,6]:
        return 2
    elif x in [7,8,9]:
        return 3
    elif x in [10,11,12]:
        return 4

In [14]:
fdata['Квартал'] = fdata['Месяц'].apply(month)

In [137]:
fdata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 124196 entries, 0 to 124195
Data columns (total 13 columns):
1C            124196 non-null object
ISBN          124190 non-null object
EAN           124175 non-null object
Тип_Данных    124196 non-null object
Месяц         124196 non-null int64
Год           124196 non-null int64
Кол-во        124196 non-null int64
Prom          124196 non-null int64
Ниша          124196 non-null object
Цена          124196 non-null float64
Автор         124077 non-null object
Название      124196 non-null object
Квартал       124196 non-null int64
dtypes: float64(1), int64(5), object(7)
memory usage: 13.3+ MB


In [15]:
n = pd.read_excel('Ниши.xlsx')
n1 = n['Ниши']
n1 = n1.to_numpy(dtype = str)

In [16]:
fdata['Ниша'] = fdata['Ниша'].apply(lambda x: x if x in n1 else 0)

In [17]:
fdata = fdata[fdata['Ниша'] != 0]

In [18]:
blog = 'Белоконь,Зубарева,Набокова,Краснова,Доктор Регина,Митрошина,Гандапас,Перл,Столярова,Мока,Енко,Беспятых,Фатеева,Раков,Соболь,Бабанова,Перцева,Доктор Вагин,Постригай,Монова,Суркова'
blog = blog.split(',')

In [19]:
b = np.array(blog)
b

array(['Белоконь', 'Зубарева', 'Набокова', 'Краснова', 'Доктор Регина',
       'Митрошина', 'Гандапас', 'Перл', 'Столярова', 'Мока', 'Енко',
       'Беспятых', 'Фатеева', 'Раков', 'Соболь', 'Бабанова', 'Перцева',
       'Доктор Вагин', 'Постригай', 'Монова', 'Суркова'], dtype='<U13')

In [20]:
fdata = fdata[['ISBN', 'Тип_Данных', 'Месяц', 'Кол-во', 'Prom', 'Ниша', 'Цена', 'Автор', 'Название']]

In [21]:
fdata.dropna(subset=['Автор'], inplace=True)

In [22]:
fdata['Автор'] = fdata['Автор'].map(lambda x: x.split(' ')[0])

In [23]:
fdata['Blog'] = fdata['Автор'].apply(lambda x: 1 if x in b or x == 'Доктор' else 0)

In [34]:
fdata.to_excel('regdata.xlsx')

In [25]:
vk = pd.read_excel('vkdata.xlsx')

In [26]:
fb = pd.read_excel('facebook.xlsx')

In [26]:
v1 = vk[vk['Дата создания'] >= '2020']
v1 = v1[['Кампания', 'Дата создания']]

In [70]:
v1.head()

,Кампания,Дата создания
0,Бот-Рекомендатор,2020-11-24
1,Кулинарка (8 книг),2020-11-23
2,BTS-маркетинг,2020-11-23
3,Тюдор,2020-11-23
4,Спортивные легенды (суммарно 30к на все книги),2020-11-16


In [27]:
vdata = pd.merge(fdata,
                v1,
                left_on = 'Автор',
                right_on = 'Кампания',
                how = 'inner')

In [28]:
vdata['Дата создания'] = vdata['Дата создания'].map(lambda x: x.split('-')[1])

In [29]:
vdata['Дата создания']= vdata['Дата создания'].map(int)

In [63]:
vdata['Camp'] = np.where([vdata['Месяц'] >= vdata['Дата создания']] == True, 1, 0)

In [65]:
vdata['Camp'].value_counts()

0    2086
Name: Camp, dtype: int64

In [32]:
prdata = vdata[vdata['Месяц'] < vdata['Дата создания']]

In [72]:
prdata['ISBN'].value_counts()

978-5-17-106077-0    47
978-5-17-111582-1    40
978-5-17-112722-0    38
978-5-17-097283-8    30
978-5-17-112547-9    27
978-5-17-122685-5    14
978-5-17-118617-3    13
978-5-17-105788-6    12
978-5-17-108775-3    10
978-5-17-127341-5     6
978-5-17-107829-4     4
978-5-17-103335-4     3
978-5-17-112887-6     2
978-5-17-119695-0     1
978-5-17-112798-5     1
978-5-17-114293-3     1
Name: ISBN, dtype: int64

In [33]:
postdata = vdata[vdata['Месяц'] >= vdata['Дата создания']]

In [73]:
postdata['ISBN'].value_counts()

978-5-17-107829-4    42
978-5-17-108775-3    42
978-5-17-114293-3    36
978-5-17-119695-0    33
978-5-17-112887-6    26
978-5-17-118617-3    21
978-5-17-112547-9    20
978-5-17-112722-0    13
978-5-17-112798-5    10
978-5-17-105788-6    10
978-5-17-106077-0    10
978-5-17-127341-5    10
978-5-17-111582-1     9
978-5-17-097283-8     5
978-5-17-122685-5     4
978-5-17-103335-4     1
Name: ISBN, dtype: int64

In [24]:
regdata = regdata[['ISBN', 'Тип_Данных', 'Месяц', 'Год', 'Кол-во', 'Prom', 'Автор', 'Цена (100 % прайс)']]

In [74]:
x = prdata['ISBN']
x = x.to_numpy(dtype = str)
y = postdata['ISBN']
y = y.to_numpy(dtype = str)

In [75]:
x = list(x)
y = list(y)

In [76]:
n = []

for book in x:
    if book in y:
        n.append(book)

In [77]:
n = list(dict.fromkeys(n))

In [78]:
prdata = prdata[prdata['ISBN'].isin(n)]

In [79]:
postdata = postdata[postdata['ISBN'].isin(n)]

In [88]:
postdata.to_excel('aprom.xlsx')
prdata.to_excel('bprom.xlsx')

In [80]:
do = prdata.groupby(by='ISBN')['Кол-во'].sum()
do.to_excel('do.xlsx')

In [81]:
posle = postdata.groupby(by='ISBN')['Кол-во'].sum()
posle.to_excel('posle.xlsx')

In [162]:
prod = fdata[fdata['Prom'] == 1]

In [163]:
prod = prod.groupby(by=['Автор', 'Название', 'Квартал', 'Тип_Данных'])['Кол-во'].sum().unstack()

In [164]:
prod.fillna(0, inplace = True)

In [109]:
#prod.reset_index(inplace=True)

In [110]:
#prod[["Book24", "Ozon", "WildBerries", "Лабиринт", "ЧитайГород"]] = prod[["Book24", "Ozon", "WildBerries", "Лабиринт", "ЧитайГород"]].apply(pd.to_numeric)

In [167]:
prod['Интернет-магазины'] = prod['Book24'] + prod['Ozon'] + prod['WildBerries'] + prod['Лабиринт']
prod['ЧГ и Буквоед'] = prod['Буквоед'] + prod['ЧитайГород']

In [168]:
dataprod = prod.sort_values(by='Интернет-магазины', ascending=False)
dataprod.to_excel('Проекты_продвижения.xlsx')

In [62]:
#fdata.groupby(by=['Ниша', 'Квартал', 'Тип_Данных'])['Кол-во'].median()

In [61]:
#pd.pivot_table(fdata, values=['Кол-во'], index=['Ниша','Квартал', 'Тип_Данных'],
                    #columns=['Prom'], aggfunc={'Кол-во':np.median}, fill_value=0).unstack(level=-1)